In [3]:
import pandas as pd
import os

# LOAD AND PREPARE THE DATA

**Definitions of every column**

Season	Match Season  
DateTime	Match Date and Time (yyyy-mm-dd hh:mm:ss)  
HomeTeam	Home Team  
AwayTeam	Away Team  
FTHG	Full Time Home Team Goals  
FTAG	Full Time Away Team Goals  
FTR	Full Time Result (H=Home Win, D=Draw, A=Away Win)  
HTHG	Half Time Home Team Goals  
HTAG	Half Time Away Team Goals  
HTR	Half Time Result (H=Home Win, D=Draw, A=Away Win)  
Referee	Match Referee  
HS	Home Team Shots  
AS	Away Team Shots  
HST	Home Team Shots on Target  
AST	Away Team Shots on Target  
HC	Home Team Corners  
AC	Away Team Corners  
HF	Home Team Fouls Committed  
AF	Away Team Fouls Committed  
HY	Home Team Yellow Cards  
AY	Away Team Yellow Cards  
HR	Home Team Red Cards  
AR	Away Team Red Cards  

In [6]:
df = pd.read_csv('results.csv', encoding='latin-1')  
print(df.head())

    Season              DateTime     HomeTeam        AwayTeam  FTHG  FTAG FTR  \
0  1993-94  1993-08-14T00:00:00Z      Arsenal        Coventry     0     3   A   
1  1993-94  1993-08-14T00:00:00Z  Aston Villa             QPR     4     1   H   
2  1993-94  1993-08-14T00:00:00Z      Chelsea       Blackburn     1     2   A   
3  1993-94  1993-08-14T00:00:00Z    Liverpool  Sheffield Weds     2     0   H   
4  1993-94  1993-08-14T00:00:00Z     Man City           Leeds     1     1   D   

   HTHG  HTAG  HTR  ... HST  AST  HC  AC  HF  AF  HY  AY  HR  AR  
0   NaN   NaN  NaN  ... NaN  NaN NaN NaN NaN NaN NaN NaN NaN NaN  
1   NaN   NaN  NaN  ... NaN  NaN NaN NaN NaN NaN NaN NaN NaN NaN  
2   NaN   NaN  NaN  ... NaN  NaN NaN NaN NaN NaN NaN NaN NaN NaN  
3   NaN   NaN  NaN  ... NaN  NaN NaN NaN NaN NaN NaN NaN NaN NaN  
4   NaN   NaN  NaN  ... NaN  NaN NaN NaN NaN NaN NaN NaN NaN NaN  

[5 rows x 23 columns]


## Overall shape/specification/details

In [ ]:
print("Dataset shape: ", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nData types:")
print(df.dtypes)
print("\nMissing values per column:")
print(df.isnull().sum())
print("\nSeasons available:")
print(df['Season'].unique())
print(f"\nDate range: {df['DateTime'].min()} to {df['DateTime'].max()}")

Dataset shape:  (11113, 23)

Column names:
['Season', 'DateTime', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR']

Missing values per column:
Season         0
DateTime       0
HomeTeam       0
AwayTeam       0
FTHG           0
FTAG           0
FTR            0
HTHG         924
HTAG         924
HTR          924
Referee     2824
HS          2824
AS          2824
HST         2824
AST         2824
HC          2824
AC          2824
HF          2824
AF          2824
HY          2824
AY          2824
HR          2824
AR          2824
dtype: int64

Seasons available:
['1993-94' '1994-95' '1995-96' '1996-97' '1997-98' '1998-99' '1999-00'
 '2000-01' '2001-02' '2002-03' '2003-04' '2004-05' '2005-06' '2006-07'
 '2007-08' '2008-09' '2009-10' '2010-11' '2011-12' '2012-13' '2013-14'
 '2014-15' '2015-16' '2016-17' '2017-18' '2018-19' '2019-20' '2020-21'
 '2021-22']

Date range: 1993-08-14T00:00:0

## Conclusion

We can observe that a lot of important data from seasons before 2000 is not available, so we have to cut out these seasons, because we will use more specific statistics than just the score. 

In [15]:
# Filter for seasons 2000 onwards
df_filtered = df[df['Season'] >= '2000-01'].copy()

print(f"Original dataset: {len(df)} matches")
print(f"Filtered dataset (2000+): {len(df_filtered)} matches")
print(f"\nSeasons included: {sorted(df_filtered['Season'].unique())}")

# Check missing values in filtered data
print("\nMissing values in filtered data:")
print(df_filtered.isnull().sum())

# Check data completeness
print(f"\nPercentage of missing values per column:")
print((df_filtered.isnull().sum() / len(df_filtered) * 100).round(2))

# Basic statistics
print(f"\nBasic stats:")
print(f"Total matches: {len(df_filtered)}")
print(f"Unique teams: {len(set(df_filtered['HomeTeam'].unique()) | set(df_filtered['AwayTeam'].unique()))}")
print(f"\nResult distribution:")
print(df_filtered['FTR'].value_counts())
print(f"\nResult percentages:")
print(df_filtered['FTR'].value_counts(normalize=True) * 100)

Original dataset: 11113 matches
Filtered dataset (2000+): 8289 matches

Seasons included: ['2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22']

Missing values in filtered data:
Season      0
DateTime    0
HomeTeam    0
AwayTeam    0
FTHG        0
FTAG        0
FTR         0
HTHG        0
HTAG        0
HTR         0
Referee     0
HS          0
AS          0
HST         0
AST         0
HC          0
AC          0
HF          0
AF          0
HY          0
AY          0
HR          0
AR          0
dtype: int64

Percentage of missing values per column:
Season      0.0
DateTime    0.0
HomeTeam    0.0
AwayTeam    0.0
FTHG        0.0
FTAG        0.0
FTR         0.0
HTHG        0.0
HTAG        0.0
HTR         0.0
Referee     0.0
HS          0.0
AS          0.0
HST         0.0
AST         0.0
HC          0.0


Perfect! Now we have the optimal dataset for the model, next we have to prepare it.

In [19]:
df_filtered['DateTime'] = pd.to_datetime(df_filtered['DateTime'])

df_filtered = df_filtered.sort_values('DateTime').reset_index(drop=True)

df_filtered['Year'] = df_filtered['DateTime'].dt.year
df_filtered['Month'] = df_filtered['DateTime'].dt.month
df_filtered['DayOfWeek'] = df_filtered['DateTime'].dt.dayofweek

print("Data prepared! Shape:", df_filtered.shape)
print(df_filtered[['DateTime', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']].head(10))

Data prepared! Shape: (8289, 26)
                   DateTime    HomeTeam       AwayTeam  FTHG  FTAG FTR
0 2000-08-19 00:00:00+00:00    Charlton       Man City     4     0   H
1 2000-08-19 00:00:00+00:00     Chelsea       West Ham     4     2   H
2 2000-08-19 00:00:00+00:00    Coventry  Middlesbrough     1     3   A
3 2000-08-19 00:00:00+00:00       Derby    Southampton     2     2   D
4 2000-08-19 00:00:00+00:00       Leeds        Everton     2     0   H
5 2000-08-19 00:00:00+00:00   Leicester    Aston Villa     0     0   D
6 2000-08-19 00:00:00+00:00   Liverpool       Bradford     1     0   H
7 2000-08-19 00:00:00+00:00  Sunderland        Arsenal     1     0   H
8 2000-08-19 00:00:00+00:00   Tottenham        Ipswich     3     1   H
9 2000-08-20 00:00:00+00:00  Man United      Newcastle     2     0   H


# Model

We need to calculate team strength (elo, like in chess) based on past matches

To do that we need to create a function that adds more columns/informations do our dataset based on the matches

In [25]:
def calculate_team_form(df, n_matches=5):

    df_with_features = df.copy()

    # Initialize feature columns
    features_to_add = [
        'home_wins_last_5', 'home_draws_last_5', 'home_losses_last_5',
        'away_wins_last_5', 'away_draws_last_5', 'away_losses_last_5',
        'home_goals_scored_avg', 'home_goals_conceded_avg',
        'away_goals_scored_avg', 'away_goals_conceded_avg',
        'home_points_last_5', 'away_points_last_5'
    ]
    
    for feature in features_to_add:
        df_with_features[feature] = 0.0

    for idx in range(len(df_with_features)):
        if idx % 500 == 0:
            print(f"Processing match {idx}/{len(df_with_features)}")
        
        current_match = df_with_features.iloc[idx]
        home_team = current_match["HomeTeam"]
        away_team = current_match["AwayTeam"]
        current_date = current_match["DateTime"]

        prev_matches = df_with_features[df_with_features['DateTime'] < current_date]

        if len(prev_matches) == 0:
            continue

        home_prev = prev_matches[
            (prev_matches['HomeTeam'] == home_team) |
            (prev_matches['AwayTeam'] == home_team)
        ].tail(n_matches)

        if len(home_prev) > 0:
            home_team_wins = 0
            home_team_draws = 0
            home_team_losses = 0
            home_team_goals_scored = []
            home_team_goals_conceded = []

            for _, match in home_prev.iterrows():
                if match['HomeTeam'] == home_team:
                    # Home team was playing at home
                    goals_scored = match['FTHG']
                    goals_conceded = match['FTAG']
                    result = match['FTR']

                    if result == 'H':
                        home_team_wins += 1
                    elif result == 'D':
                        home_team_draws += 1
                    else:
                        home_team_losses += 1
                
                else:
                    # Home team was playing away
                    goals_scored = match['FTAG']
                    goals_conceded = match['FTHG']
                    result = match['FTR']

                    if result == 'A':
                        home_team_wins += 1
                    elif result == 'D':
                        home_team_draws += 1
                    else:
                        home_team_losses += 1

                home_team_goals_scored.append(goals_scored)
                home_team_goals_conceded.append(goals_conceded)
            
            # Store gained data 
            df_with_features.at[idx, 'home_wins_last_5'] = home_team_wins
            df_with_features.at[idx, 'home_losses_last_5'] = home_team_losses
            df_with_features.at[idx, 'home_draws_last_5'] = home_team_draws
            df_with_features.at[idx, 'home_goals_scored_avg'] = sum(home_team_goals_scored)/len(home_team_goals_scored)
            df_with_features.at[idx, 'home_goals_conceded_avg'] = sum(home_team_goals_conceded)/len(home_team_goals_conceded)
            df_with_features.at[idx, 'home_points_last_5'] = home_team_wins*3 + home_team_draws



        away_prev = prev_matches[
            (prev_matches['HomeTeam'] == away_team) |
            (prev_matches['AwayTeam'] == away_team)
        ].tail(n_matches)


        if len(away_prev) > 0:
            away_team_wins = 0
            away_team_draws = 0
            away_team_losses = 0
            away_team_goals_scored = []
            away_team_goals_conceded = []
            
            for _, match in away_prev.iterrows():
                if match['HomeTeam'] == away_team:
                    goals_scored = match['FTHG']
                    goals_conceded = match['FTAG']
                    result = match['FTR']
                    
                    if result == 'H':
                        away_team_wins += 1
                    elif result == 'D':
                        away_team_draws += 1
                    else:
                        away_team_losses += 1
                else:
                    goals_scored = match['FTAG']
                    goals_conceded = match['FTHG']
                    result = match['FTR']
                    
                    if result == 'A':
                        away_team_wins += 1
                    elif result == 'D':
                        away_team_draws += 1
                    else:
                        away_team_losses += 1
                
                away_team_goals_scored.append(goals_scored)
                away_team_goals_conceded.append(goals_conceded)
            
            # Store away team features
            df_with_features.at[idx, 'away_wins_last_5'] = away_team_wins
            df_with_features.at[idx, 'away_draws_last_5'] = away_team_draws
            df_with_features.at[idx, 'away_losses_last_5'] = away_team_losses
            df_with_features.at[idx, 'away_goals_scored_avg'] = sum(away_team_goals_scored) / len(away_team_goals_scored)
            df_with_features.at[idx, 'away_goals_conceded_avg'] = sum(away_team_goals_conceded) / len(away_team_goals_conceded)
            df_with_features.at[idx, 'away_points_last_5'] = away_team_wins * 3 + away_team_draws
    
    print("Feature engineering complete!")
    return df_with_features

## Feature Engineering Function

This function calculates rolling team performance statistics based on their last N matches. For each match in the dataset, it looks back at historical data (only matches that occurred before the current match) and computes form indicators for both the home and away teams.

### Features Calculated (per team):
- **Wins/Draws/Losses**: Count of results in last 5 matches
- **Points**: Total points earned (3 for win, 1 for draw, 0 for loss)
- **Goals Scored Average**: Mean goals scored per match
- **Goals Conceded Average**: Mean goals conceded per match

### Key Implementation Details:
- **No lookahead bias**: Only uses data from matches that occurred before the current match date
- **Handles both home and away**: Tracks team performance regardless of venue
- **Flexible window**: Can adjust `n_matches` parameter (default is 5 matches)
- **Skips early matches**: Matches without sufficient history are left with default values (0.0)

### Parameters:
- `df`: DataFrame containing match results with columns: DateTime, HomeTeam, AwayTeam, FTHG, FTAG, FTR
- `n_matches`: Number of previous matches to consider for rolling statistics (default: 5)

### Returns:
DataFrame with original columns plus 12 new feature columns (6 for home team, 6 for away team)

### Usage:
```python
df_features = calculate_team_form(df_filtered, n_matches=5)
```

### Note:
This function can take several minutes to run on large datasets as it processes each match sequentially to maintain temporal integrity.

In [27]:
df_features = calculate_team_form(df_filtered, n_matches=5)

# Check if any features are non-zero
print("\nSum of home_points_last_5:", df_features['home_points_last_5'].sum())
print("Max of home_points_last_5:", df_features['home_points_last_5'].max())

print("\n Features created:")
print(df_features[['HomeTeam', 'AwayTeam', 'FTR', 'home_points_last_5', 
                    'away_points_last_5', 'home_goals_scored_avg', 
                    'away_goals_scored_avg']].tail(20))

Processing match 0/8289
Processing match 500/8289
Processing match 1000/8289
Processing match 1500/8289
Processing match 2000/8289
Processing match 2500/8289
Processing match 3000/8289
Processing match 3500/8289
Processing match 4000/8289
Processing match 4500/8289
Processing match 5000/8289
Processing match 5500/8289
Processing match 6000/8289
Processing match 6500/8289
Processing match 7000/8289
Processing match 7500/8289
Processing match 8000/8289
Feature engineering complete!

Sum of home_points_last_5: 55677.0
Max of home_points_last_5: 15.0

 Features created:
            HomeTeam        AwayTeam FTR  home_points_last_5  \
8269          Wolves     Aston Villa   H                 6.0   
8270           Leeds     Southampton   D                 6.0   
8271         Chelsea       Brentford   A                15.0   
8272        Brighton         Norwich   D                 0.0   
8273         Burnley        Man City   A                 4.0   
8274      Man United       Leicester   D   